# Import

In [12]:
import os
import sys
from datetime import datetime
import time

import re

import tqdm
import itertools

import json
import urllib.request

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import lxml


# URL Rule

In [13]:
# comp_id = 380725    # 기업 고유 UID
p_num = 1             # 리뷰 페이지 (4 reviews / page)

### query로 CompID 가져오기

In [14]:
Keyword = '넷마블'

In [15]:
search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
search_req = requests.get(search_url.format(Keyword))
# print(search_req.text)

In [16]:
search_soup = BeautifulSoup(search_req.text, 'lxml')
# search_soup.text

In [17]:
search_soup.select('li:nth-child(1) > div.txt > p.name > a')

[<a href="/Comp/CompSummary/H79411">
                 넷마블
             </a>]

In [18]:
comp_id = re.findall('/([^"]*)"', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))[0].split('/')[2]
comp_name = re.findall('>([^"]*)<', re.sub(r'\s', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a'))))[0]

comp_id, comp_name

('H79411', '넷마블')

In [19]:
URL = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=5000&isNew=false&employType=0&isEmploy=false&jobCode='
print(URL)

https://www.catch.co.kr/api/v1.0/comp/reviewInfo/H79411/commentList?currentPage=1&pageSize=5000&isNew=false&employType=0&isEmploy=false&jobCode=


# Json to DataFrame 

In [20]:
# Json 형식의 웹 데이터 가져오기
raw_data = requests.get(URL).json()
# print(raw_data)

In [21]:
data = raw_data['reviewList']
data[0]

{'idx': 207128,
 'CompID': 'H79411',
 'CompName': '넷마블',
 'RegDate': '2023-06-08T10:46:41.900Z',
 'CI': 'H79411.jpg',
 'EmployText': '전직',
 'Gender2': 2,
 'RecomName': '추천',
 'EmployType': '계약직',
 'NewOld': '경력',
 'Answer': None,
 'Good': '눈치볼필요없다   복장자유롭고 자유로운분위기 \n 복리후생 굿   ',
 'Bad': '동종업계대비 낮은연봉 업무 스트레스 다소높음\n 고인물이 많음  ',
 'UsefulY': 0,
 'MyUsefulY': None,
 'MyOpinion': None,
 'CareerYear': 5,
 'CareerYearYN': 'Y',
 'Keyword1': None,
 'Keyword2': None,
 'Keyword3': None,
 'Keyword1YN': 'N',
 'Keyword2YN': 'N',
 'Keyword3YN': 'N',
 'JobName': '일반영업',
 'Area': '서울',
 'MyStarScore': 4}

In [22]:
df = pd.DataFrame(data)
df.tail()

,idx,CompID,CompName,RegDate,CI,EmployText,Gender2,RecomName,EmployType,NewOld,...,CareerYearYN,Keyword1,Keyword2,Keyword3,Keyword1YN,Keyword2YN,Keyword3YN,JobName,Area,MyStarScore
16,141843,H79411,넷마블,2021-04-27T12:52:03.703Z,H79411.jpg,전직,2,비추,정규직,신입,...,Y,None,None,None,N,N,N,네트워크/서버/보안,서울,3.2
17,127605,H79411,넷마블,2020-11-18T13:16:38.293Z,H79411.jpg,현직,2,추천,정규직,경력,...,Y,None,None,None,N,N,N,QA/테스트/검증,None,3.0
18,122637,H79411,넷마블,2020-11-17T08:33:04.327Z,H79411.jpg,현직,2,추천,정규직,경력,...,Y,None,None,None,N,N,N,기획/전략/경영,None,4.0
19,121616,H79411,넷마블,2020-10-18T11:16:33.700Z,H79411.jpg,현직,2,비추,정규직,신입,...,Y,None,None,None,N,N,N,디자인기타,None,3.2
20,121474,H79411,넷마블,2020-10-11T13:09:45.607Z,H79411.jpg,전직,2,추천,정규직,경력,...,Y,None,None,None,N,N,N,그래픽디자인/CG,None,4.0


In [23]:
df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'CI', 'EmployText', 'Gender2',
       'RecomName', 'EmployType', 'NewOld', 'Answer', 'Good', 'Bad', 'UsefulY',
       'MyUsefulY', 'MyOpinion', 'CareerYear', 'CareerYearYN', 'Keyword1',
       'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN',
       'JobName', 'Area', 'MyStarScore'],
      dtype='object')

In [24]:
df.drop(['CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 'CareerYear', 'CareerYearYN',
            'MyUsefulY', 'RecomName', 'MyOpinion', 'CareerYear', 'Keyword1', 
            'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], 
            axis=1, inplace=True)

df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'EmployText', 'Good', 'Bad',
       'JobName', 'MyStarScore'],
      dtype='object')

In [25]:
# 정규식으로 이스케이프 문자 제거        # Good, Bad 리뷰 항목에만 적용.
df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df.tail()

,idx,CompID,CompName,RegDate,EmployText,Good,Bad,JobName,MyStarScore
16,141843,H79411,넷마블,2021-04-27T12:52:03.703Z,전직,"복장과 문화가 자유로워요, 다들 열심히 일하는 분위기, 구디에 있는 것도 장점",야근이 너무 많아요. 상사 잘못 걸리면 무능한 상사 밑에서 노가다만 해야해요,네트워크/서버/보안,3.2
17,127605,H79411,넷마블,2020-11-18T13:16:38.293Z,현직,시간 연차 등 비교적 시간을 자유롭게 쓸 수 있다.,회사가 오래되어서 모든 것이 노후화 되어 있다.,QA/테스트/검증,3.0
18,122637,H79411,넷마블,2020-11-17T08:33:04.327Z,현직,자유로운 점이 좋습니다. 복지도 나쁘지 않습니다. 야근은 팀바팀인데 저희는 없어서 ...,구로에 위치해서 교통이 애매하고 면적대비 인구비율(?)이 높은 편인것 같습니다.,기획/전략/경영,4.0
19,121616,H79411,넷마블,2020-10-18T11:16:33.700Z,현직,"네임밸류, 나쁘지 않은 연봉 테이블, 요즘 야근 안함","커리어를 신경 써주지 않는 회사, 구내식당 없음",디자인기타,3.2
20,121474,H79411,넷마블,2020-10-11T13:09:45.607Z,전직,게임업계에서 자기계발을 할 수 있는 딱딱하지 않은 몇 안되는 회사,특별히 없지만 커리어를 위해 본인 업무에 최선을 다해야 합니다,그래픽디자인/CG,4.0


In [26]:
# RegDate 컬럼의 데이터 - 문자열 9번째 까지 출력 (YYYYMMDD 형식의 날짜 표기)
df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())
df.tail()

,idx,CompID,CompName,RegDate,EmployText,Good,Bad,JobName,MyStarScore
16,141843,H79411,넷마블,20210427,전직,"복장과 문화가 자유로워요, 다들 열심히 일하는 분위기, 구디에 있는 것도 장점",야근이 너무 많아요. 상사 잘못 걸리면 무능한 상사 밑에서 노가다만 해야해요,네트워크/서버/보안,3.2
17,127605,H79411,넷마블,20201118,현직,시간 연차 등 비교적 시간을 자유롭게 쓸 수 있다.,회사가 오래되어서 모든 것이 노후화 되어 있다.,QA/테스트/검증,3.0
18,122637,H79411,넷마블,20201117,현직,자유로운 점이 좋습니다. 복지도 나쁘지 않습니다. 야근은 팀바팀인데 저희는 없어서 ...,구로에 위치해서 교통이 애매하고 면적대비 인구비율(?)이 높은 편인것 같습니다.,기획/전략/경영,4.0
19,121616,H79411,넷마블,20201018,현직,"네임밸류, 나쁘지 않은 연봉 테이블, 요즘 야근 안함","커리어를 신경 써주지 않는 회사, 구내식당 없음",디자인기타,3.2
20,121474,H79411,넷마블,20201011,전직,게임업계에서 자기계발을 할 수 있는 딱딱하지 않은 몇 안되는 회사,특별히 없지만 커리어를 위해 본인 업무에 최선을 다해야 합니다,그래픽디자인/CG,4.0


# For 문으로 만들어보기
    - 리뷰 데이터가 있는 페이지까지 반복

In [27]:
rv = []

for p_num in range(1, 52) :
    url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=4&isNew=false&employType=0&isEmploy=false&jobCode='
    rv.append(requests.get(url).json()['reviewList'])


data = list(itertools.chain.from_iterable(rv))
df = pd.DataFrame(data)


df.drop(['CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 'CareerYear', 
            'RecomName', 'CareerYearYN', 'MyUsefulY', 'MyOpinion', 'Area',
            'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN'], 
            axis=1, inplace=True)
df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:6].strip())


df.tail()

,idx,CompID,CompName,RegDate,EmployText,Good,Bad,JobName,MyStarScore
16,141843,H79411,넷마블,202104,전직,"복장과 문화가 자유로워요, 다들 열심히 일하는 분위기, 구디에 있는 것도 장점",야근이 너무 많아요. 상사 잘못 걸리면 무능한 상사 밑에서 노가다만 해야해요,네트워크/서버/보안,3.2
17,127605,H79411,넷마블,202011,현직,시간 연차 등 비교적 시간을 자유롭게 쓸 수 있다.,회사가 오래되어서 모든 것이 노후화 되어 있다.,QA/테스트/검증,3.0
18,122637,H79411,넷마블,202011,현직,자유로운 점이 좋습니다. 복지도 나쁘지 않습니다. 야근은 팀바팀인데 저희는 없어서 ...,구로에 위치해서 교통이 애매하고 면적대비 인구비율(?)이 높은 편인것 같습니다.,기획/전략/경영,4.0
19,121616,H79411,넷마블,202010,현직,"네임밸류, 나쁘지 않은 연봉 테이블, 요즘 야근 안함","커리어를 신경 써주지 않는 회사, 구내식당 없음",디자인기타,3.2
20,121474,H79411,넷마블,202010,전직,게임업계에서 자기계발을 할 수 있는 딱딱하지 않은 몇 안되는 회사,특별히 없지만 커리어를 위해 본인 업무에 최선을 다해야 합니다,그래픽디자인/CG,4.0


# While 문으로 만들기 
    - 그 다음 페이지에 리뷰 데이터 개수가 0 이면 멈춤


In [ ]:
# len(requests.get(url).json()['reviewList'])

In [28]:
# Catch 사이트에 검색을 하면, 최상위 검색결과의 comp_ui를 받아오기

comp = '넷마블'
search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
search_req = requests.get(search_url.format(comp)) # Keyword = comp_name
# print(search_req.text)

search_soup = BeautifulSoup(search_req.text, 'lxml')
# search_soup.text
search_soup.select('li:nth-child(1) > div.txt > p.name > a')

comp_id = re.findall('/([^"]*)"', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))[0].split('/')[2]
comp_name = re.findall('>([^"]*)<', re.sub(r'\s', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a'))))[0]

comp_id, comp_name

('H79411', '넷마블')

In [31]:
# 위에서 받아온 comp_id를 가지고 데이터 받아와서 데이터프레임으로 만들기

try : 
    # comp_id = 380725
    p_num = 1

    rv = []


    while True : 

        url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=2000&isNew=false&employType=0&isEmploy=false&jobCode='
        rv.append(requests.get(url).json()['reviewList'])

        p_num += 1

        if len(requests.get(url).json()['reviewList']) == 0 :
            break


    data = list(itertools.chain.from_iterable(rv))      # = 리스트 컴프리헨션
    df = pd.DataFrame(data)


    df.drop(['CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 'CareerYear', 
            'RecomName', 'CareerYearYN', 'MyUsefulY', 'MyOpinion', 'Area',
            'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN'], 
            axis=1, inplace=True)
            
    df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
    df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
    df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:6].strip())
    


except :
    print("리뷰가 존재하지 않습니다.")


df.tail()


,idx,CompID,CompName,RegDate,EmployText,Good,Bad,JobName,MyStarScore
16,141843,H79411,넷마블,202104,전직,"복장과 문화가 자유로워요, 다들 열심히 일하는 분위기, 구디에 있는 것도 장점",야근이 너무 많아요. 상사 잘못 걸리면 무능한 상사 밑에서 노가다만 해야해요,네트워크/서버/보안,3.2
17,127605,H79411,넷마블,202011,현직,시간 연차 등 비교적 시간을 자유롭게 쓸 수 있다.,회사가 오래되어서 모든 것이 노후화 되어 있다.,QA/테스트/검증,3.0
18,122637,H79411,넷마블,202011,현직,자유로운 점이 좋습니다. 복지도 나쁘지 않습니다. 야근은 팀바팀인데 저희는 없어서 ...,구로에 위치해서 교통이 애매하고 면적대비 인구비율(?)이 높은 편인것 같습니다.,기획/전략/경영,4.0
19,121616,H79411,넷마블,202010,현직,"네임밸류, 나쁘지 않은 연봉 테이블, 요즘 야근 안함","커리어를 신경 써주지 않는 회사, 구내식당 없음",디자인기타,3.2
20,121474,H79411,넷마블,202010,전직,게임업계에서 자기계발을 할 수 있는 딱딱하지 않은 몇 안되는 회사,특별히 없지만 커리어를 위해 본인 업무에 최선을 다해야 합니다,그래픽디자인/CG,4.0


# 컬럼명 및 데이터 변경

### 컬럼명 변경

In [32]:
df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'EmployText', 'Good', 'Bad',
       'JobName', 'MyStarScore'],
      dtype='object')

In [33]:
df.rename(columns=
            {'idx':'review_uid', 
            'CompID':'comp_uid',
            'CompName':'comp_name', 
            'RegDate':'review_date', 
            'EmployText':'is_office', 
            'Good':'review_pos', 
            'Bad':'review_neg', 
            'MyStarScore':'review_rate', 
            'JobName':'position'}, 
            inplace=True)

df.head(1)

,review_uid,comp_uid,comp_name,review_date,is_office,review_pos,review_neg,position,review_rate
0,207128,H79411,넷마블,202306,전직,눈치볼필요없다 복장자유롭고 자유로운분위기 복리후생 굿,동종업계대비 낮은연봉 업무 스트레스 다소높음 고인물이 많음,일반영업,4.0


### 전/현직 여부를 Boolean 값으로 변경

In [35]:
df[['is_office']].tail()

,is_office
16,전직
17,현직
18,현직
19,현직
20,전직


In [36]:
df['is_office'] = df['is_office'].apply(lambda x: x.replace('현직', '1').strip() == '1')
df[['is_office']].tail()

,is_office
16,False
17,True
18,True
19,True
20,False


### review_rate 의 값들을 전부 정수형으로 반올림

In [ ]:
df['review_rate'].round(0).astype(int)[:5]

In [37]:
df['review_rate'] = df['review_rate'].round(0).astype(int)
df.tail()

,review_uid,comp_uid,comp_name,review_date,is_office,review_pos,review_neg,position,review_rate
16,141843,H79411,넷마블,202104,False,"복장과 문화가 자유로워요, 다들 열심히 일하는 분위기, 구디에 있는 것도 장점",야근이 너무 많아요. 상사 잘못 걸리면 무능한 상사 밑에서 노가다만 해야해요,네트워크/서버/보안,3
17,127605,H79411,넷마블,202011,True,시간 연차 등 비교적 시간을 자유롭게 쓸 수 있다.,회사가 오래되어서 모든 것이 노후화 되어 있다.,QA/테스트/검증,3
18,122637,H79411,넷마블,202011,True,자유로운 점이 좋습니다. 복지도 나쁘지 않습니다. 야근은 팀바팀인데 저희는 없어서 ...,구로에 위치해서 교통이 애매하고 면적대비 인구비율(?)이 높은 편인것 같습니다.,기획/전략/경영,4
19,121616,H79411,넷마블,202010,True,"네임밸류, 나쁘지 않은 연봉 테이블, 요즘 야근 안함","커리어를 신경 써주지 않는 회사, 구내식당 없음",디자인기타,3
20,121474,H79411,넷마블,202010,False,게임업계에서 자기계발을 할 수 있는 딱딱하지 않은 몇 안되는 회사,특별히 없지만 커리어를 위해 본인 업무에 최선을 다해야 합니다,그래픽디자인/CG,4


### 긍/부정 리뷰들을 한 컬럼에 합치기

In [41]:
a = df.drop(['review_pos'], axis=1).rename(columns = {'review_neg':'review_cont'})
b = df.drop(['review_neg'], axis=1).rename(columns = {'review_pos':'review_cont'})

new_df = pd.concat([a, b], ignore_index = True)

new_df.tail()

,review_uid,comp_uid,comp_name,review_date,is_office,review_cont,position,review_rate
37,141843,H79411,넷마블,202104,False,"복장과 문화가 자유로워요, 다들 열심히 일하는 분위기, 구디에 있는 것도 장점",네트워크/서버/보안,3
38,127605,H79411,넷마블,202011,True,시간 연차 등 비교적 시간을 자유롭게 쓸 수 있다.,QA/테스트/검증,3
39,122637,H79411,넷마블,202011,True,자유로운 점이 좋습니다. 복지도 나쁘지 않습니다. 야근은 팀바팀인데 저희는 없어서 ...,기획/전략/경영,4
40,121616,H79411,넷마블,202010,True,"네임밸류, 나쁘지 않은 연봉 테이블, 요즘 야근 안함",디자인기타,3
41,121474,H79411,넷마블,202010,False,게임업계에서 자기계발을 할 수 있는 딱딱하지 않은 몇 안되는 회사,그래픽디자인/CG,4


# DataFrame to CSV file

In [42]:
SAVE_PATH = r'./data/'

In [43]:
df['comp_name'][0]

'넷마블'

In [44]:
comp = df['comp_name'][0]

In [45]:
# 디렉토리 생성 (이미 존재하면 생성하지 않음)

os.makedirs(SAVE_PATH, exist_ok=True)

# csv 파일로 저장.

file_name = f"{comp}_catch.csv"
save_file_path = os.path.join(SAVE_PATH, file_name)

df.to_csv(save_file_path, index=False, encoding = "utf-8")

In [47]:
pd.read_csv(save_file_path)

,review_uid,comp_uid,comp_name,review_date,is_office,review_pos,review_neg,position,review_rate
0,207128,H79411,넷마블,202306,False,눈치볼필요없다 복장자유롭고 자유로운분위기 복리후생 굿,동종업계대비 낮은연봉 업무 스트레스 다소높음 고인물이 많음,일반영업,4
1,206247,H79411,넷마블,202306,True,다양한 경험을 쌓을 수 있음. 조직문화가 딱딱하지 않아 적응이 쉽다.,근무강도가 힘든 편이다. 혼자 다양한 일을 하다보니 업무가 과중하다,포장/가공,4
2,201336,H79411,넷마블,202305,False,회사 내에서 자유롭고 일하기도 편한 곳이다 상사와도 편안한 관계로 지낼 수 있고 눈...,개인적인 능력이 딸리면 일하기가 힘들고 대부분이 경력직이며 신입들은 거의 없는 수준...,그래픽디자인/CG,4
3,200208,H79411,넷마블,202305,True,눈치볼 필요없이 칼퇴근가능하다. 워라벨이 아주 좋다. 만족,보수적이고 수직적인 분위기로 경직되고 수동적인 업무형태,게임,5
4,200105,H79411,넷마블,202305,False,네임밸류와 자유로운 것이 장점 나름 인천에서 출근하기 편함,상사가 능력이 모자르고 편애가 심함 팀장 자격이 있어서보다는 할 사람이 없어서,경리/회계/결산,4
5,197115,H79411,넷마블,202305,False,대기업. 초봉 높은 편. 프로그래머는 연봉 잘 줌. 복지포인트 나옴. 여러 프로젝트...,"맨날 사람 자르고 동료들 팀 바꿔 대고, 10분 단위로 사람 감시함. 야근 엄청 많음",캐릭터/만화/애니,1
6,193234,H79411,넷마블,202304,True,퍼블리셔다 보니 어느 게임 회사보다 다양한 프로젝트 수행 가능(사업부도 로테이션 종...,마케팅 부서는 업무 강도가 정말 정말 쎄다. 모든 산업군 다 합쳐도 넷마블 마케팅실...,마케팅/PR/분석,2
7,192458,H79411,넷마블,202303,True,출퇴근 자유로움 연차사용 자유로움 건물이 깨끗해요(새건물),회사위치가 어느 역에서든 다 멀어요 걸어서 20분 고인물이 회사분위기 다 흐립니다,경리/회계/결산,1
8,192267,H79411,넷마블,202303,True,자유로운 유연근무 개인주의 강해 회식이나 행사 거의 없음,"경영진이 성과나 회사의 방향성, 비전을 공유해주지 않음 성과급, 연봉 인상률 등의 ...",기획/전략/경영,3
9,185725,H79411,넷마블,202210,False,눈치볼 필요가 없이 칼퇴가 가능하다. 워라벨이 가능하며 급여도 정기적으로 인상이 된...,가끔 야근이 필요하며 파벌이 있고 퇴근 후에도 게임 지식을 쌓아야 적응할 수 있다.,일반영업,2


# 함수화

In [48]:
import os
import sys
from datetime import datetime
import time

import re

import tqdm
import itertools

import json
import urllib.request

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import lxml

In [49]:
SAVE_PATH = r'./data/'
# SAVE_PATH = r'/app/data/reviews/'     <--- for Docker

In [50]:
def get_review(comp = str, save = False): 
    
    search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
    search_req = requests.get(search_url.format(comp)) # Keyword = comp_name
    # print(search_req.text)

    search_soup = BeautifulSoup(search_req.text, 'lxml')
    # search_soup.text
    search_soup.select('li:nth-child(1) > div.txt > p.name > a')

    comp_id = re.findall('/([^"]*)"', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))[0].split('/')[2]
    comp_name = re.findall('>([^"]*)<', re.sub(r'\s', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a'))))[0]




    p_num = 1

    rv = []



    try : 

        while True : 

            url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=5000&isNew=false&employType=0&isEmploy=false&jobCode='
            rv.append(requests.get(url).json()['reviewList'])

            p_num += 1

            if len(requests.get(url).json()['reviewList']) == 0 :
                break


        data = list(itertools.chain.from_iterable(rv))
        df = pd.DataFrame(data)


        df.drop(['idx', 'CompID', 'CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 
                'RecomName', 'CareerYearYN', 'MyUsefulY', 'MyOpinion', 'Area', 'CareerYear',
                'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN'], 
                axis=1, inplace=True)
        

        df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
        df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())

        df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:6].strip())

        df['EmployText'] = df['EmployText'].apply(lambda x: x.replace('현직', '1').strip() == '1')



        df.rename(columns=
           {'RegDate':'review_date', 
           'EmployText':'is_office', 
           'Good':'review_pos', 
           'Bad':'review_neg', 
           'MyStarScore':'review_rate', 
           'JobName':'position'
           }, inplace=True)


        df['review_rate'] = df['review_rate'].round(0).astype(int)



        # 긍/부정 리뷰들만 찢어서 합치기

        a = df.drop(['review_pos'], axis=1).rename(columns = {'review_neg':'review_cont'})
        b = df.drop(['review_neg'], axis=1).rename(columns = {'review_pos':'review_cont'})

        new_df = pd.concat([a, b], ignore_index = True)

        # return new_df




        # csv 파일로 저장.

        file_name = f"{comp_name}_catch.csv"
        save_file_path = os.path.join(SAVE_PATH, file_name)


        if save is True :  
            os.makedirs(SAVE_PATH, exist_ok=True)
            new_df.to_csv(save_file_path, index=False, encoding = "utf-8")
            print('결과를 저장했습니다.')
            return pd.read_csv(save_file_path)
        
        else : 
            print('결과가 저장되지 않습니다.')
            return new_df

        
    except : 
        print("리뷰가 존재하지 않습니다.")




In [51]:
get_review('펄어비스', True)

결과를 저장했습니다.


,CompName,review_date,is_office,review_cont,position,review_rate
0,펄어비스,202305,False,업무 스타일은 호불호가 갈릴 수 있으며 야근 시 야근비 따로 없음,마케팅/PR/분석,3
1,펄어비스,202212,True,회사 전반적으로 비전에 대한 공유가 아쉽고 경력 개발에 대한 기회가 부족하다고 생각함,기획/전략/경영,2
2,펄어비스,202207,False,"최고경영자는 바뀐지 얼마 안돼 모르겠고, 실장급 그이상 리더십들 중 정상적으로 경력...",기획/전략/경영,2
3,펄어비스,202103,False,야근이 매우 많음. 굳이 필요없는 야근도 많은 듯한 느낌이 듬. 고용 불안정,QA/테스트/검증,4
4,펄어비스,202305,False,정기적으로 명절 떡값이 제공되며 연령대가 높지 않고 연차 자유 사용,마케팅/PR/분석,3
5,펄어비스,202212,True,신사옥으로 이전하면서 휴게시설과 복지시설들은 만족스러운 편임,기획/전략/경영,2
6,펄어비스,202207,False,복지 좋다. 비포괄임금이다. 밥이 맛있는편이다. 주거지원비때문에 출퇴근이 편하다,기획/전략/경영,2
7,펄어비스,202103,False,"교통 편리, 직원 복지 및 사내 시설이 좋음 카페나 휴식 공간 등 쾌적해요.",QA/테스트/검증,4


In [52]:
get_review('화인석재')

리뷰가 존재하지 않습니다.


In [53]:
get_review('삼성전자')

결과가 저장되지 않습니다.


,CompName,review_date,is_office,review_cont,position,review_rate
0,삼성전자,202308,True,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4
1,삼성전자,202308,False,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3
2,삼성전자,202306,False,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,5
3,삼성전자,202306,True,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,3
4,삼성전자,202306,False,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2
...,...,...,...,...,...,...
797,삼성전자,202009,False,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,판매/캐셔/매장관리,4
798,삼성전자,202009,True,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,기획/전략/경영,4
799,삼성전자,202009,False,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,재무/세무/IR,4
800,삼성전자,202009,False,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,네트워크/서버/보안,3


In [54]:
get_review('초록뱀미디어')

결과가 저장되지 않습니다.


,CompName,review_date,is_office,review_cont,position,review_rate
0,초록뱀미디어,202104,True,일도 제대로 안하고 위에 사람들은 무얼 하는지 모르겠다. 인간성 제로,영상/사진/촬영,1
1,초록뱀미디어,202104,True,좋은점이라고는 없다. 그냥 다녀보면 안다.. 답답 그자체,영상/사진/촬영,1


In [ ]:
get_review('카카오')

In [55]:
from new_catch import get_review

In [56]:
get_review('와이지엔터')

결과가 저장되지 않습니다.


,CompName,review_date,is_office,review_cont,position,review_rate
0,와이지엔터테인먼트,202306,False,신입으로 들어오면 고생만한다. 경력직으로 입사하면 많은 걸 누린다. 돈도 조금 주면...,음악/음향,1
1,와이지엔터테인먼트,202305,False,근무 시간이 없다. 언제 어디서라도 업무를 할 준비가 되어있어야 하는 곳,마케팅/PR/분석,1
2,와이지엔터테인먼트,202104,False,꼰대가 너무 많다. 줄타기 실패하면 망한다. 본받을수없는 상사가 많다,공연/엔터테인먼트,4
3,와이지엔터테인먼트,202101,True,워라벨을 중시한다면 다시 한번 고려해보세요.,마케팅/PR/분석,2
4,와이지엔터테인먼트,202011,True,"연봉이 매우매우매우 낮다, 딱히 직무능력이라고 할 수 있는 게 없어서 성장하기 어렵다",마케팅/PR/분석,2
5,와이지엔터테인먼트,202011,False,"인턴, 계약직은 너무 불안정해서 코로나 터지고 위태해요",의류/패션/잡화디자인,3
6,와이지엔터테인먼트,202011,True,일이 너무 많고 사건사고가 너무 많다. 일 터지면 골치 너무 아프다,전시/컨벤션,3
7,와이지엔터테인먼트,202011,True,음...블랙핑크 말고 돈 줄이 없다. 발전가능성 낮음,기획/전략/경영,2
8,와이지엔터테인먼트,202306,False,점심 저녁 준다. 위치가 좋다. 복장이 자유롭다. 쌀국수가 맛있다. 연자 높을수록 ...,음악/음향,1
9,와이지엔터테인먼트,202305,False,그저 네임밸류? 대중들이 열광하는 콘텐츠가 있다는 것? 그 외에는...,마케팅/PR/분석,1


In [58]:
get_review('제이와이피', True)

결과를 저장했습니다.


,CompName,review_date,is_office,review_cont,position,review_rate
0,제이와이피엔터테인먼트,202306,False,워라밸은 거의 없었고요. 연예인을 보조해주는 자리이다보니 쉽지않습니다. 그만큼 칼같...,시설/보안/경비/안전,4
1,제이와이피엔터테인먼트,202305,True,업무강도가 몰릴때 확실하게 몰려서 내 생활 찾기가 어려움. 초과근무를 해도 수당이 없음,경리/회계/결산,4
2,제이와이피엔터테인먼트,202305,True,야근이 많다. 업무강도가 쎄다. 육아휴직이 없다. 집에서 멀다,마케팅/PR/분석,5
3,제이와이피엔터테인먼트,202210,True,워라밸 보장이 안 되고 계속 곡 완성 될 때까지 야근 해야함,음악/음향,5
4,제이와이피엔터테인먼트,202110,False,모든 곳이 그렇듯 꼰대가 있습니다,마케팅/PR/분석,4
5,제이와이피엔터테인먼트,202110,False,"쉴 때도 살짝 눈치를 보는 편이고, 일 할때도 조금 불안해요.",인쇄/출판/편집,3
6,제이와이피엔터테인먼트,202012,False,"최악의 워라밸, 본보기가 되는 상사가 적습니다.",마케팅/PR/분석,2
7,제이와이피엔터테인먼트,202011,False,식당 노쇼일 경우 추가금 내야하는 점이 별로였습니다.,마케팅/PR/분석,4
8,제이와이피엔터테인먼트,202011,True,일하는 강도에 비해 연봉이 너무 적고 복지가 부족하다,마케팅/PR/분석,3
9,제이와이피엔터테인먼트,202011,False,"체계가 없다, 근속연수가 짧은 편이다, 이직시 동종업계가 아니면 경력인정 어렵다",공연/엔터테인먼트,3
